In [ ]:
%%bigquery --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.ai_works_wide AS (
  SELECT DISTINCT a.work_id, a.doi,
    IF(d.country='Brazil'
      OR d.country='Brasil'
      OR d.country_code='BR',True,False) AS br,
    d.country, d.country_code,
    a.topic_id,
  FROM (
    SELECT w.id AS work_id, w.doi, w.type,
      ai.topic_id AS topic_id
    FROM `insyspo.publicdb_openalex_2025_08_rm.works` AS w
    JOIN `insyspo.publicdb_openalex_2025_08_rm.works_topics` AS wt
    ON w.id = wt.work_id
    JOIN projectdb_ciaam.ai_topics_curated AS ai
    ON wt.topic_id = ai.topic_id
    WHERE wt.score>0.95
  ) AS a
  JOIN publicdb_openalex_2025_08_rm.works_authorships AS c
  ON a.work_id = c.work_id
  JOIN publicdb_openalex_2025_08_rm.institutions AS d
  ON c.institution_id = d.id
  WHERE a.type = 'article'# AND
  #  (
  #    d.country_code = 'BR'
  #    OR d.country = 'Brazil'
  #    OR d.country = 'Brasil'
  #  )
)

In [17]:
%%bigquery df --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.pesquisadores_ia_lattes AS (
  SELECT a.*
  FROM `insyspo.projectdb_lattes_2025.total_TextosinicialfinalCV` AS a
  JOIN `insyspo.projectdb_lattes_2025.total_Tabelaconsolidada` AS b
  ON a.IDLattes = b.IDLattes
  WHERE (LOWER(ResumoCV) LIKE '%inteligência artificial%'
    OR LOWER(ResumoCV) LIKE 'machine learning'
    OR LOWER(ResumoCV) LIKE '%artificial intelligence%'
    OR LOWER(ResumoCV) LIKE '%aprendizado de máquina%'
    OR LOWER(ResumoCV) LIKE '%aprendizado profundo%'
    OR LOWER(ResumoCV) LIKE '%deep learning%')
    AND b.Graduacao >= '1'
)

Query is running:   0%|          |

In [13]:
%%bigquery df --project=insyspo

CREATE OR REPLACE TABLE projectdb_ciaam.pesquisadores_ia_comunidades AS (
  SELECT b.display_name, a.*, c.IDLattes
  FROM (
    SELECT author_id, area, COUNT(DISTINCT work_id) AS works
    FROM projectdb_ciaam.all_sources_works_authors
    GROUP BY author_id, area
  ) AS a
  JOIN publicdb_openalex_2025_08_rm.authors AS b
  ON a.author_id = b.id
  JOIN projectdb_lattes.match_researchers_lattes_openalex AS c
  ON a.author_id = c.author_id
  JOIN projectdb_lattes_2025.total_Tabelaconsolidada AS e
  ON c.IDLattes = e.IDLattes
  WHERE EndProfPais = 'Brasil' AND Graduacao >= '1' #AND works_topic>=3
  ORDER BY works DESC
)

Query is running:   0%|          |

In [18]:
%%bigquery df --project=insyspo

SELECT a.*, b.*, c.*
FROM projectdb_ciaam.pesquisadores_ia_lattes AS a
FULL JOIN projectdb_ciaam.br_authors_br_ai_works_wide AS b
ON a.IDLattes = b.IDLattes
FULL JOIN projectdb_ciaam.pesquisadores_ia_comunidades AS c
ON a.IDLattes = c.IDLattes


Query is running:   0%|          |

Downloading:   0%|          |

In [19]:
df.head(50)

,IDLattes,Nome,ResumoCV,Outrasinformacoesrelevantes,display_name,author_id,works,IDLattes_1,display_name_1,author_id_1,area,works_1,IDLattes_2
0,'1056435898334313,André Luiz Monteiro de Oliveira,Possui graduação em Direito pela Universidade ...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
1,'1005222224619536,André Thomaz Gandolpho de Mello,Tem experiência na área de Ciência da Computaç...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
2,'0994029202983034,Aline Medeiros Ferreira de Araujo,Mestra em Engenharia de Construção Civil pela ...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
3,'0988318226549616,Carlos Henrique Campos Silva,Possui graduação em Sistema de Informação pelo...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
4,'0981699993777790,Giovani Manica Barili,Academicamente é Mestre em computação aplicada...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
5,'0981656712227882,Edgard Braz Alves,Mestre em Engenharia da Computação no IME-RJ (...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
6,'0978837964647017,Ana Lucia Cetertich Bazzan,Graduou-se em Engenharia pela Universidade de ...,None,Ana L. C. Bazzan,5028225369,32,'0978837964647017,Ana L. C. Bazzan,5028225369,Multi-agentes,6,'0978837964647017
7,'0978837964647017,Ana Lucia Cetertich Bazzan,Graduou-se em Engenharia pela Universidade de ...,None,Ana L. C. Bazzan,5028225369,32,'0978837964647017,Ana L. C. Bazzan,5028225369,Neural,5,'0978837964647017
8,'0929751972236350,Anibal Evaristo Fernandes,Graduado em Engenharia Elétrica pela Faculdade...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None
9,'0916863398304561,Alison Roger Hajo Weber,Possui graduação em Tecnologia em Análise e De...,None,None,<NA>,<NA>,None,None,<NA>,None,<NA>,None


In [20]:
df.to_excel('tabela_combinada_pesquisadores_ia_br.xlsx')

In [21]:
%%bigquery df --project=insyspo

SELECT a.*, b.*, c.*
FROM projectdb_ciaam.pesquisadores_ia_lattes AS a
JOIN projectdb_ciaam.br_authors_br_ai_works_wide AS b
ON a.IDLattes = b.IDLattes
JOIN projectdb_ciaam.pesquisadores_ia_comunidades AS c
ON a.IDLattes = c.IDLattes

Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
df.head(20)

,IDLattes,Nome,ResumoCV,Outrasinformacoesrelevantes,display_name,author_id,works,IDLattes_1,display_name_1,author_id_1,area,works_1,IDLattes_2
0,'0978837964647017,Ana Lucia Cetertich Bazzan,Graduou-se em Engenharia pela Universidade de ...,None,Ana L. C. Bazzan,5028225369,32,'0978837964647017,Ana L. C. Bazzan,5028225369,Multi-agentes,6,'0978837964647017
1,'0978837964647017,Ana Lucia Cetertich Bazzan,Graduou-se em Engenharia pela Universidade de ...,None,Ana L. C. Bazzan,5028225369,32,'0978837964647017,Ana L. C. Bazzan,5028225369,Neural,5,'0978837964647017
2,'8728806963872778,Diego Barreto Haddad,Possui doutorado em engenharia elétrica pela C...,None,Diego B. Haddad,5047733466,12,'8728806963872778,Diego B. Haddad,5047733466,Neural,5,'8728806963872778
3,'8120721917914721,Samuel da Silva Oliveira,Possui graduação em Redes de Computadores pela...,None,Samuel da Silva Oliveira,5020377291,1,'8120721917914721,Samuel da Silva Oliveira,5020377291,Neural,1,'8120721917914721
4,'7807664442719582,Kaike Sa Teles Rocha Alves,Gestor de ciência de dados na PGE-PR e doutora...,None,Kaike Sa Teles Rocha Alves,5045912974,11,'7807664442719582,Kaike Sa Teles Rocha Alves,5045912974,Fuzzy,2,'7807664442719582
5,'8879153966426815,Marcos Roberto e Souza,Doutor em Ciência da Computação pela Universid...,None,Marcos Roberto e Souza,5012863628,8,'8879153966426815,Marcos Roberto e Souza,5012863628,Visão,1,'8879153966426815
6,'6377657274398145,Danilo Sipoli Sanches,Possui graduação em Ciência da Computação pela...,None,Danilo Sipoli Sanches,5088676977,5,'6377657274398145,Danilo Sipoli Sanches,5088676977,Neural,1,'6377657274398145
7,'5453814076903614,Bruno Werneck Pinto Hoelz,Possui graduação em Bacharelado em Ciência da ...,None,Bruno W. P. Hoelz,5030637412,2,'5453814076903614,Bruno W. P. Hoelz,5030637412,Multi-agentes,1,'5453814076903614
8,'4524723055652545,Ajalmar Rêgo da Rocha Neto,Ajalmar Rocha Neto é mestre (2006) e doutor (2...,None,Ajalmar R. Rocha Neto,5071945694,18,'4524723055652545,Ajalmar R. Rocha Neto,5071945694,Neural,3,'4524723055652545
9,'0427387583450747,Ricardo Araújo Rios,Ricardo possui doutorado em Ciência da Computa...,None,Ricardo A. Rios,5010287469,4,'0427387583450747,Ricardo A. Rios,5010287469,Fuzzy,3,'0427387583450747


In [23]:
df.to_excel('tabela_interseccao_pesquisadores_ia_br.xlsx')

In [24]:
df.shape

(599, 13)